In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

from src.models import train_arima, create_lstm_sequences, build_lstm
from src.metrics import mae, rmse, mape


# Load Processed Tesla Data

In [ ]:
tsla = pd.read_csv(
    "data/processed/TSLA_processed.csv",
    index_col=0,
    parse_dates=True
)

tsla.head()


# Train / Test Split (Chronological)

In [ ]:
train = tsla.loc[: "2024-12-31"]
test = tsla.loc["2025-01-01":]

train_close = train["Adj Close"]
test_close = test["Adj Close"]

print("Train size:", len(train))
print("Test size:", len(test))


# ARIMA Model

In [ ]:
arima_model = train_arima(train_close, order=(5, 1, 0))

arima_forecast = arima_model.forecast(steps=len(test_close))


# ARIMA Evaluation

In [ ]:
arima_mae = mae(test_close.values, arima_forecast.values)
arima_rmse = rmse(test_close.values, arima_forecast.values)
arima_mape = mape(test_close.values, arima_forecast.values)

print("ARIMA Performance")
print("MAE:", arima_mae)
print("RMSE:", arima_rmse)
print("MAPE:", arima_mape)


# ARIMA Visualization

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(train_close.index, train_close, label="Train")
plt.plot(test_close.index, test_close, label="Test")
plt.plot(test_close.index, arima_forecast, label="ARIMA Forecast")
plt.legend()
plt.title("ARIMA Forecast vs Actual")
plt.show()


# Prepare Data for LSTM

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))

scaled_data = scaler.fit_transform(
    tsla[["Adj Close"]]
)

train_size = len(train)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size - 60:]


# Create LSTM Sequences

In [ ]:
X_train, y_train = create_lstm_sequences(train_data, window_size=60)
X_test, y_test = create_lstm_sequences(test_data, window_size=60)

print(X_train.shape, y_train.shape)


# Build & Train LSTM

In [ ]:
lstm_model = build_lstm((X_train.shape[1], 1))

history = lstm_model.fit(
    X_train,
    y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)


# LSTM Forecast

In [ ]:
lstm_predictions = lstm_model.predict(X_test)

lstm_predictions = scaler.inverse_transform(lstm_predictions)
y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))


# LSTM Evaluation

In [ ]:
lstm_mae = mae(y_test_actual, lstm_predictions)
lstm_rmse = rmse(y_test_actual, lstm_predictions)
lstm_mape = mape(y_test_actual, lstm_predictions)

print("LSTM Performance")
print("MAE:", lstm_mae)
print("RMSE:", lstm_rmse)
print("MAPE:", lstm_mape)


# LSTM Visualization

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(test_close.index, test_close.values, label="Actual")
plt.plot(test_close.index, lstm_predictions.flatten(), label="LSTM Forecast")
plt.legend()
plt.title("LSTM Forecast vs Actual")
plt.show()


# Model Comparison Table

In [ ]:
comparison = pd.DataFrame({
    "Model": ["ARIMA", "LSTM"],
    "MAE": [arima_mae, lstm_mae],
    "RMSE": [arima_rmse, lstm_rmse],
    "MAPE": [arima_mape, lstm_mape]
})

comparison
